# Imports

In [109]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import random

# Constants

In [110]:
torch.manual_seed(1337)
block_size = 32
batch_size = 256
split_pct = 0.9
max_iters = 1500
eval_interval = 500
learning_rate = 1e-3 #3e-4
eval_iters = 200
n_embd = 64 #384
n_layer = 1 #4 # Number of block layers
n_head = 1 #4
dropout = 0.2

In [111]:
# Set GPU device
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
print(device)

mps


# Dataloader

In [112]:
chars = ['\\', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', '-', '.', '=', 'e']
vocab_size = len(chars)
print("Number of tokens: {}".format(vocab_size))

Number of tokens: 16


In [113]:
encoder = { ch:i for i,ch in enumerate(chars) }
decoder = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [encoder[c] if c in chars else print (s) for c in s]
decode = lambda l: ''.join([decoder[i] for i in l])

In [135]:
def get_batch(bs):
    numbers = torch.randint(0, 1000000, (bs, 2))
    x = []
    y = []
    for ix in range(bs):
        # Create equation with solution digits reversed
        equation = "{}+{}={}".format(numbers[ix,0], numbers[ix,1], str((numbers[ix,0] + numbers[ix,1]).item()))
        # Pick a random place in equation after = and try to predict the next character
        string_idx = torch.randint(equation.index("=")+1, len(equation), (1,))
        # Add padding so input and target both have lengths equal to block size
        x.append(torch.tensor(encode("\\" * (block_size - string_idx) + equation[:string_idx])))
        y.append(torch.tensor(encode("\\" * (block_size - string_idx - 2) + equation[:string_idx+1] + "\\")))
    return torch.stack(x).to(device), torch.stack(y).to(device)

# Model

In [116]:
# Obtains average loss over multiple batches
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, T = get_batch()
            logits, loss = m(X, T)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [117]:
class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.c_attn = nn.Linear(n_embd, 3 * n_embd, bias=False) # (C, 3C)
        self.register_buffer('tril', torch.tril(torch.ones(1, 1, block_size, block_size)))
        self.dropout1 = nn.Dropout(dropout)
        self.proj = nn.Linear(n_embd, n_embd) # (C, C)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x):
        # x has shape (B,T,C) - because of initial embeddings
        B, T, C = x.shape

        q, k, v = self.c_attn(x).split(n_embd, dim=2) # (B, T, C) @ (C, 3C) --> (B, T, 3C) splits -> (B, T, C)
        k = k.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)
        q = q.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)
        v = v.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)

        # affinities
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, nh, T, hs) @ (B, nh, hs, T) ---> (B,nh,T,T)
        #wei = wei.masked_fill(self.tril[:, :, :T, :T] == 0, float('-inf')) # (B, nh, T, T)
        wei = F.softmax(wei, dim=-1) # (B, nh, T, T)
        wei = self.dropout1(wei) # (B, nh, T, T)
        out = wei @ v # (B, nh, T, T) @ (B, nh, T, hs) --> (B, nh, T, hs)
        out = out.transpose(1, 2).contiguous().view(B,T,C) # (B, T, nh, hs) -> (B, T, C)
        out = self.dropout2(self.proj(out)) # (B, T, C) @ (C, C) --> (B, T, C)
        return out

In [118]:
class FeedForward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # Projection layer
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [119]:
class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        self.sa = MultiHeadAttention(n_head)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [120]:
class GptModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # Final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tkn_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tkn_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets, ignore_index=encode("\\")[0])
            
        return logits, loss

    def generate(self, no_sums):
        # idx is (B, T) array of indices in the current context
        idx, _ = get_batch(no_sums)
        max_new_tokens = 24
        for _ in range(max_new_tokens):
            # crop idx to last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get predictions
            logits, loss = self(idx_cond)
            # focus only on last time step
            logits = logits[:, -1, :]
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # )B, C)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [121]:
m = GptModel()
m.to(device)

GptModel(
  (token_embedding_table): Embedding(16, 64)
  (position_embedding_table): Embedding(32, 64)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (c_attn): Linear(in_features=64, out_features=192, bias=False)
        (dropout1): Dropout(p=0.2, inplace=False)
        (proj): Linear(in_features=64, out_features=64, bias=True)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=64, out_features=256, bias=True)
          (1): ReLU()
          (2): Linear(in_features=256, out_features=64, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
  )
  (ln_f): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (lm_head): Linear(in_features=64, out_features=16, bias=True)
)

In [122]:
# Number of parameters in model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

0.054032 M parameters


# Training

In [123]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

In [124]:
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch(batch_size)
    
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 2.8773, val loss 2.8764
step 500: train loss 0.1241, val loss 0.1239
step 1000: train loss 0.1160, val loss 0.1166
step 1500: train loss 0.1154, val loss 0.1156
step 2000: train loss 0.1152, val loss 0.1148
step 2500: train loss 0.1150, val loss 0.1151
step 3000: train loss 0.1148, val loss 0.1145
step 3500: train loss 0.1148, val loss 0.1144
step 4000: train loss 0.1142, val loss 0.1139
step 4500: train loss 0.1150, val loss 0.1147
step 4999: train loss 0.1140, val loss 0.1137


# Inference

In [125]:
print(decode(m.generate(no_sums=10)[0].tolist()))

# Save the model

In [ ]:
#import time
#torch.save(m.state_dict(), './models/addition_calculator_{}.pt'.format(time.strftime("%Y-%m-%d-%H:%M")))